## A more complex example: python + rust

Let's use current PoC to build a project that depends indirectly on rust since this seems
to be the most common use case (and, quite frankly, it's the use case I'm particularly
most interested).

For this purpose I chose to use discovery upstream code (quipucords), since I'm familiar
with its build process. It's a python project that depends on cryptography and bcrypt -
libraries that have rust dependencies.

In [1]:
# lets start cloning both quipucords
!git clone git@github.com:quipucords/quipucords.git --depth 1 --branch=1.2.5

Cloning into 'quipucords'...
remote: Enumerating objects: 460, done.
remote: Counting objects: 100% (460/460), done.
remote: Compressing objects: 100% (374/374), done.
remote: Total 460 (delta 51), reused 199 (delta 26), pack-reused 0
Receiving objects: 100% (460/460), 871.99 KiB | 1.82 MiB/s, done.
Resolving deltas: 100% (51/51), done.
Note: switching to '4d91ee52124fcf1d3a2b5d725043f57957e73356'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false



Quipucords don't have it's rust dependencies listed like python dependencies. To fix 
this, let's create a quick script to search it's requirements/requirements-build.

In the future we could have a improved version of this script and recommend that users
run it (like `pip_find_builddeps.py` script).

In [2]:
from pathlib import Path
import tarfile

# this script requires you to pip install pybuild-deps, which I built as an alternative
# to pip_find_builddeps
from pybuild_deps.find_build_dependencies import find_build_dependencies
from pybuild_deps.get_package_source import get_package_source

from cachi2.core.package_managers.pip import PipRequirementsFile

LOCK_FILE = "Cargo.lock"
PACKAGE_FILE = "Cargo.toml"

def depends_on_rust(deps: list) -> bool:
    for d in deps:
        if d.startswith("setuptools-rust"):
            return True
    return False

def _save_dep_file(tarball, full_file_name, path_to_dependencies_file):
    file_handle = tarball.extractfile(full_file_name)
    path_to_dependencies_file.parent.mkdir(exist_ok=True, parents=True)
    path_to_dependencies_file.write_bytes(file_handle.read())

def find_rust_deps(requirements_file):
    req_file = PipRequirementsFile(Path(requirements_file))
    for req in req_file.requirements:
        package_name = req.package
        version_qualifier, version = req.version_specs[0]
        assert version_qualifier == "==", "an EXACT match is required."
        build_dependencies = find_build_dependencies(package_name, version)
        if not depends_on_rust(build_dependencies):
            continue
        source_code = get_package_source(package_name, version)
        with tarfile.open(fileobj=source_code.open("rb")) as tarball:
            for full_file_name in tarball.getnames():
                # this is assuming a source code contains only one cargo.toml/lock, 
                # which might not be true - (for the packages used in this exercise it 
                # is OK)
                if full_file_name.endswith((LOCK_FILE, PACKAGE_FILE)):
                    file_name = Path(full_file_name).name
                    path_to_dependencies_file = (
                        req_file.file_path.parent / "dependencies" / f"{package_name}-rust" / file_name
                    )
                    _save_dep_file(tarball, full_file_name, path_to_dependencies_file)

In [3]:
find_rust_deps("quipucords/requirements.txt")
find_rust_deps("quipucords/requirements-build.txt")

In [4]:
!tree quipucords/dependencies

quipucords/dependencies
├── bcrypt-rust
│   ├── Cargo.lock
│   └── Cargo.toml
└── cryptography-rust
    ├── Cargo.lock
    └── Cargo.toml

3 directories, 4 files


## Pre-fetch dependencies

Now with all requirements files in place we can use cachi2 to fetch dependencies

In [5]:
!cachi2 fetch-deps --source quipucords '[{"type": "pip"},{"type": "cargo", "path": "dependencies/cryptography-rust"},{"type": "cargo", "path": "dependencies/bcrypt-rust"}]'

2023-04-28 14:32:58,091 INFO Extracting metadata from setup.py
2023-04-28 14:32:58,093 INFO Found name in setup.py: 'quipucords'
2023-04-28 14:32:58,094 INFO Found version in setup.py: '0.0.0'
2023-04-28 14:32:58,094 INFO Resolved package name: 'quipucords'
2023-04-28 14:32:58,094 INFO Resolved package version: '0.0.0'
2023-04-28 14:32:58,126 INFO No hash options used, will not require hashes unless HTTP(S) dependencies are present.
2023-04-28 14:32:58,127 INFO Downloading amqp==5.1.1 ; python_version >= "3.9" and python_version < "4.0"
2023-04-28 14:32:58,307 INFO Successfully downloaded amqp==5.1.1 ; python_version >= "3.9" and python_version < "4.0" to deps/pip/amqp-5.1.1.tar.gz
2023-04-28 14:32:58,307 INFO Downloading ansible-core==2.13.8 ; python_version >= "3.9" and python_version < "4.0"
2023-04-28 14:32:59,406 INFO Successfully downloaded ansible-core==2.13.8 ; python_version >= "3.9" and python_version < "4.0" to deps/pip/ansible-core-2.13.8.tar.gz
2023-04-28 14:32:59,406 INFO

## Generate environment variables

In [6]:
!cachi2 generate-env "$(realpath ./cachi2-output)" -o $(pwd)/cachi2.env --for-output-dir /tmp/cachi2-output
!cat cachi2.env

export PIP_FIND_LINKS=/tmp/cachi2-output/deps/pip
export PIP_NO_INDEX=true


## Inject project files

This part is not implemented for the cargo integration yet.
We need a configuration file and a index directory so cargo can use the prefetched dependencies.

In [7]:
%%writefile cargo_config.toml
[source.crates-io]
replace-with = "local"

[source.local]
local-registry = "/tmp/cachi2-output"


Overwriting cargo_config.toml


In [8]:
# generating cargo indexes should be done only for the downloaded packages - 
# but for now lets skip that (I still don't know how to do it XD) and just clone the 
# whole thing
!git clone git@github.com:rust-lang/crates.io-index.git --depth 1 cachi2-output/index

Cloning into 'cachi2-output/index'...
remote: Enumerating objects: 137868, done.
remote: Counting objects: 100% (137868/137868), done.
remote: Compressing objects: 100% (55464/55464), done.
remote: Total 137868 (delta 74911), reused 127894 (delta 68744), pack-reused 0
Receiving objects: 100% (137868/137868), 71.82 MiB | 10.32 MiB/s, done.
Resolving deltas: 100% (74911/74911), done.
Updating files: 100% (112641/112641), done.


In [9]:
!cachi2 inject-files "$(realpath ./cachi2-output)" --for-output-dir /tmp/cachi2-output

## Build the base and UI images

In [10]:
%%writefile Containerfile.baseimage
FROM registry.redhat.io/ubi8/ubi

ARG BUILDREQUIRES='cargo gcc libffi-devel libpq-devel libtool openssl-devel patch python39-devel rust rust-std-static llvm-libs'
RUN dnf install \
        git \
        glibc-langpack-en \
        jq \
        libpq \
        make \
        openssh-clients \
        python39 \
        sshpass \
        tar \
        which \
        $BUILDREQUIRES \
        -y &&\
    dnf clean all

Overwriting Containerfile.baseimage


In [11]:
!podman build --tag quipucords-base-image -f Containerfile.baseimage .

STEP 1/3: FROM registry.redhat.io/ubi8/ubi
STEP 2/3: ARG BUILDREQUIRES='cargo gcc libffi-devel libpq-devel libtool openssl-devel patch python39-devel rust rust-std-static llvm-libs'
--> Using cache fc1e4e9a054e934f33d2f078cfe908b95e3118189cf5cd9a42e1ae1825f44d3a
--> fc1e4e9a054e
STEP 3/3: RUN dnf install         git         glibc-langpack-en         jq         libpq         make         openssh-clients         python39         sshpass         tar         which         $BUILDREQUIRES         -y &&    dnf clean all
--> Using cache a5454992f1fc76faa79b37b7a8d58109af1377496b44da84565a0d4894241bb4
COMMIT quipucords-base-image
--> a5454992f1fc
Successfully tagged localhost/quipucords-base-image:latest
a5454992f1fc76faa79b37b7a8d58109af1377496b44da84565a0d4894241bb4


quipucords also has a UI. let's build it separately and ignore cachi2 for the sake 
of simplicity

In [12]:
!git clone git@github.com:quipucords/quipucords-ui.git --depth 1 --branch=1.2.0

Cloning into 'quipucords-ui'...
remote: Enumerating objects: 543, done.
remote: Counting objects: 100% (543/543), done.
remote: Compressing objects: 100% (494/494), done.
remote: Total 543 (delta 98), reused 274 (delta 41), pack-reused 0
Receiving objects: 100% (543/543), 589.31 KiB | 1.24 MiB/s, done.
Resolving deltas: 100% (98/98), done.
Note: switching to '6ff43b4e466bd75de36f4533d4e0255f9a1b47b9'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false



In [13]:
%%writefile Containerfile.quipucords-ui
FROM registry.access.redhat.com/ubi8/nodejs-16

USER root
RUN npm --userconfig .npmrc install --global yarn
# Install/build UI
COPY quipucords-ui /app
WORKDIR /app

RUN yarn install &&\
    yarn build:brand

Overwriting Containerfile.quipucords-ui


In [14]:
!podman build --tag quipucords-ui -f Containerfile.quipucords-ui .

STEP 1/6: FROM registry.access.redhat.com/ubi8/nodejs-16
STEP 2/6: USER root
--> Using cache 2b5a30209585694c84a5eef77d9050d16c96e9cbd99cd3ccd5c5175855ed0fe4
--> 2b5a30209585
STEP 3/6: RUN npm --userconfig .npmrc install --global yarn
--> Using cache e3bdf0ef69c3de2a4f96f2596e52aee23b10ec928e7e8779374cd607713ffc58
--> e3bdf0ef69c3
STEP 4/6: COPY quipucords-ui /app
--> 01b3b4858943
STEP 5/6: WORKDIR /app
--> 283af5c87d65
STEP 6/6: RUN yarn install &&    yarn build:brand
yarn install v1.22.19
[1/5] Validating package.json...
[2/5] Resolving packages...
[3/5] Fetching packages...
[4/5] Linking dependencies...
warning " > bootstrap@5.1.3" has unmet peer dependency "@popperjs/core@^2.10.2".
warning " > patternfly-react@2.40.0" has incorrect peer dependency "react@^16.3.2".
warning " > patternfly-react@2.40.0" has incorrect peer dependency "react-dom@^15.6.2 || ^16.2.0".
warning "patternfly-react > react-bootstrap-typeahead@3.4.3" has incorrect peer dependency "react@^0.14.0 || ^15.2.0 || ^1

## Build the application image

In [15]:
%%writefile Containerfile

# FROM quipucords-ui:latest as yarn_builder

FROM quipucords-base-image:latest

# setup python virtualenv
RUN python3 -m venv /opt/venv
ENV PATH="/opt/venv/bin:${PATH}"

WORKDIR /app
# copy cargo config (this should become part of cargo inject IF crates are available 
# from local files - only problem is this file needs to be at an specific location)
# NOTICE the config is placed on root directory - this was required to successfully
# build indirect rust dependencies (cryptography and bcrypt). Looks like pip is running
# not running the build under /app
COPY cargo_config.toml /.cargo/config.toml
COPY quipucords/requirements.txt .


RUN source /tmp/cachi2.env && \
    pip install -U pip 'setuptools<67' wheel &&\
    # build/install kombu first, as it requires an older setuptools and dont follow PEP-517
    pip install kombu &&\
    pip install -U setuptools &&\
    pip install -r requirements.txt

# copy quipucords source
COPY quipucords /app
# copy UI code
COPY --from=quipucords-ui:latest /app/dist/client quipucords/client
COPY --from=quipucords-ui:latest /app/dist/templates quipucords/quipucords/templates

ENV DJANGO_DEBUG=False
ENV LANG=C
ENV LC_ALL=C
ENV PRODUCTION=True
ENV PYTHONPATH=/app/quipucords

RUN make server-static

EXPOSE 443
COPY quipucords/deploy /deploy
COPY quipucords/deploy/ssl/ /etc/ssl/qpc/

CMD ["/bin/bash", "/deploy/docker_run.sh"]


Overwriting Containerfile


In [16]:
!podman build . \
  --volume "$(realpath ./cachi2-output)":/tmp/cachi2-output:Z \
  --volume "$(realpath ./cachi2.env)":/tmp/cachi2.env:Z \
  --network none \
  --tag quipucords-no-network

STEP 1/20: FROM quipucords-base-image:latest
STEP 2/20: RUN python3 -m venv /opt/venv
--> Using cache d9185e6f5c2fdfd47fe6c943907b170ce6b3b03ace40d93e43a64b1e3b67e728
--> d9185e6f5c2f
STEP 3/20: ENV PATH="/opt/venv/bin:${PATH}"
--> Using cache d67fe7085a7c7ece58f8601fe1eed6107fdc2d5b0cc6c1399aaf88696b4dc267
--> d67fe7085a7c
STEP 4/20: WORKDIR /app
--> Using cache 021917a443648249d499c6fac3f0fd3d3829a519026694e921dc1c12f8ce0289
--> 021917a44364
STEP 5/20: COPY cargo_config.toml /.cargo/config.toml
--> Using cache 00d1a1cd976da0dfecdbe8202ed4fe6f3ebd5ec259cc048701b41f95db2ab56d
--> 00d1a1cd976d
STEP 6/20: COPY quipucords/requirements.txt .
--> Using cache 08e1ba082e883e1f3599048312811dcba882c35b8dc52e30259ec4497a274363
--> 08e1ba082e88
STEP 7/20: RUN source /tmp/cachi2.env &&     pip install -U pip 'setuptools<67' wheel &&    pip install kombu &&    pip install -U setuptools &&    pip install -r requirements.txt
--> Using cache ffcac5df9f0544f7a22db8b6608358aba746c42df6c43840fc1ad6936f5c